In [1]:
import numpy as np
import pandas as pd

from __future__ import print_function
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
# Read Dataset
df = pd.read_csv('kdd-multiclass-dataset.csv')
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [3]:
df.isna()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494015,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
494016,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
494017,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
494018,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
df = df.dropna()

**Coverting Categorical from String into Numeric Data.**

Protocol_Type tcp: 1, udp: 2, icmp: 3

Service private: 1, ftp_data: 2, eco_i: 3, telnet: 4, etc...

Flag REJ: 1, SF: 2, RSTO: 3, etc...

In [5]:
# Protocol Type Conversion
df["protocol_type"]=df["protocol_type"].replace({"tcp":1})
df["protocol_type"]=df["protocol_type"].replace({"udp":2})
df["protocol_type"]=df["protocol_type"].replace({"icmp":3})

In [6]:
# Creating new columns for numeric codes of Service and Flag
df['serv_code'] = pd.factorize(df['service'])[0] + 1
df['flag_code'] = pd.factorize(df['flag'])[0] + 1

In [7]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,serv_code,flag_code
0,0,1,http,SF,181,5450,0,0,0,0,...,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal,1,1
1,0,1,http,SF,239,486,0,0,0,0,...,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal,1,1
2,0,1,http,SF,235,1337,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal,1,1
3,0,1,http,SF,219,1337,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal,1,1
4,0,1,http,SF,217,2032,0,0,0,0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal,1,1


In [8]:
mapping = {'ipsweep': 'Probe','satan': 'Probe','nmap': 'Probe','portsweep': 'Probe','saint': 'Probe','mscan': 'Probe',
        'teardrop': 'DoS','pod': 'DoS','land': 'DoS','back': 'DoS','neptune': 'DoS','smurf': 'DoS','mailbomb': 'DoS',
        'udpstorm': 'DoS','apache2': 'DoS','processtable': 'DoS',
        'perl': 'U2R','loadmodule': 'U2R','rootkit': 'U2R','buffer_overflow': 'U2R','xterm': 'U2R','ps': 'U2R',
        'sqlattack': 'U2R','httptunnel': 'U2R',
        'ftp_write': 'R2L','phf': 'R2L','guess_passwd': 'R2L','warezmaster': 'R2L','warezclient': 'R2L','imap': 'R2L',
        'spy': 'R2L','multihop': 'R2L','named': 'R2L','snmpguess': 'R2L','worm': 'R2L','snmpgetattack': 'R2L',
        'xsnoop': 'R2L','xlock': 'R2L','sendmail': 'R2L',
        'normal': 'Normal'
        }


In [9]:
df['attack_class'] = df['label'].apply(lambda v: mapping[v])

**Splitting the Dataset into Training and Testing Dataset**

Here, we have considered the ratio of 60:40 to split the dataset into train and test dataset respectively.

We can clearly observe that the dataset is splitted into train and test data with a ratio of 60:40

In [14]:
training_data = df.sample(frac=0.6, random_state=25)
testing_data = df.drop(training_data.index)

In [15]:
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 296412
No. of testing examples: 197608


**Converting Attack Types into Multi Classification 0 and 1**

In [16]:
df["attack_class"]=df["attack_class"].replace({"DoS":1})
df["attack_class"]=df["attack_class"].replace({"U2R":2})
df["attack_class"]=df["attack_class"].replace({"R2L":3})
df["attack_class"]=df["attack_class"].replace({"Probe":4})
df["attack_class"]=df["attack_class"].replace({"Normal":5})

In [17]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,serv_code,flag_code,attack_class
0,0,1,http,SF,181,5450,0,0,0,0,...,0.11,0.0,0.0,0.0,0.0,0.0,normal,1,1,5
1,0,1,http,SF,239,486,0,0,0,0,...,0.05,0.0,0.0,0.0,0.0,0.0,normal,1,1,5
2,0,1,http,SF,235,1337,0,0,0,0,...,0.03,0.0,0.0,0.0,0.0,0.0,normal,1,1,5
3,0,1,http,SF,219,1337,0,0,0,0,...,0.03,0.0,0.0,0.0,0.0,0.0,normal,1,1,5
4,0,1,http,SF,217,2032,0,0,0,0,...,0.02,0.0,0.0,0.0,0.0,0.0,normal,1,1,5


**Normalizing the Features of our Training Dataset**

In [19]:
import numpy as np
from sklearn import preprocessing
x = df.drop(["label", "service", "flag", "attack_class"], axis=1)
y= df["attack_class"]
y=y.astype('int')
normalized_arr = preprocessing.normalize(x)
print(normalized_arr)

[[0.00000000e+00 1.83384219e-04 3.31925437e-02 ... 0.00000000e+00
  1.83384219e-04 1.83384219e-04]
 [0.00000000e+00 1.84373540e-03 4.40652760e-01 ... 0.00000000e+00
  1.84373540e-03 1.84373540e-03]
 [0.00000000e+00 7.36287997e-04 1.73027679e-01 ... 0.00000000e+00
  7.36287997e-04 7.36287997e-04]
 ...
 [0.00000000e+00 8.04035169e-04 1.63219139e-01 ... 0.00000000e+00
  8.04035169e-04 8.04035169e-04]
 [0.00000000e+00 7.92909963e-04 2.30736799e-01 ... 0.00000000e+00
  7.92909963e-04 7.92909963e-04]
 [0.00000000e+00 7.81573245e-04 1.71164541e-01 ... 0.00000000e+00
  7.81573245e-04 7.81573245e-04]]


In [20]:
norm_train_df=pd.DataFrame(normalized_arr,y)
norm_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
attack_class,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.000183,0.033193,0.999444,0.0,0.0,0.0,0.0,0.0,0.000183,...,0.000183,0.0,0.000020,0.0,0.0,0.0,0.0,0.0,0.000183,0.000183
5,0.0,0.001844,0.440653,0.896055,0.0,0.0,0.0,0.0,0.0,0.001844,...,0.001844,0.0,0.000092,0.0,0.0,0.0,0.0,0.0,0.001844,0.001844
5,0.0,0.000736,0.173028,0.984417,0.0,0.0,0.0,0.0,0.0,0.000736,...,0.000736,0.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000736,0.000736
5,0.0,0.000737,0.161508,0.986011,0.0,0.0,0.0,0.0,0.0,0.000737,...,0.000737,0.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000737,0.000737
5,0.0,0.000489,0.106126,0.993766,0.0,0.0,0.0,0.0,0.0,0.000489,...,0.000489,0.0,0.000010,0.0,0.0,0.0,0.0,0.0,0.000489,0.000489


In [21]:
norm_train_df.columns = x.columns
norm_train_df.head()

,duration,protocol_type,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,serv_code,flag_code
attack_class,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.000183,0.033193,0.999444,0.0,0.0,0.0,0.0,0.0,0.000183,...,0.000183,0.0,0.000020,0.0,0.0,0.0,0.0,0.0,0.000183,0.000183
5,0.0,0.001844,0.440653,0.896055,0.0,0.0,0.0,0.0,0.0,0.001844,...,0.001844,0.0,0.000092,0.0,0.0,0.0,0.0,0.0,0.001844,0.001844
5,0.0,0.000736,0.173028,0.984417,0.0,0.0,0.0,0.0,0.0,0.000736,...,0.000736,0.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000736,0.000736
5,0.0,0.000737,0.161508,0.986011,0.0,0.0,0.0,0.0,0.0,0.000737,...,0.000737,0.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000737,0.000737
5,0.0,0.000489,0.106126,0.993766,0.0,0.0,0.0,0.0,0.0,0.000489,...,0.000489,0.0,0.000010,0.0,0.0,0.0,0.0,0.0,0.000489,0.000489


In [22]:
X_train, X_test, y_train, y_test = train_test_split(normalized_arr,y,train_size=0.6,test_size=0.4, random_state=42)

In [23]:
# Feature Selection
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [24]:
sel.get_support()

array([False,  True, False,  True, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False, False, False,  True,
       False, False,  True,  True,  True, False, False, False, False,
       False, False, False,  True,  True])

In [25]:
selected_feat= norm_train_df.columns[(sel.get_support())]
len(selected_feat)

11

In [26]:
print(selected_feat)

Index(['protocol_type', 'dst_bytes', 'logged_in', 'count', 'srv_count',
       'same_srv_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'serv_code', 'flag_code'],
      dtype='object')


In [27]:
# Shape X_train data
X_train.shape

(296412, 41)

In [28]:
X_train = X_train[:,sel.get_support()]
X_test = X_test[:,sel.get_support()]


In [29]:
# Model
model = Sequential()
model.add(LSTM(4,input_dim=11))  # try using a GRU
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
print(model.get_config())

{'name': 'sequential', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, None, 11), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'lstm_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'LSTM', 'config': {'name': 'lstm', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, None, 11), 'return_sequences': False, 'return_state': False, 'go_backwards': False, 'stateful': False, 'unroll': False, 'time_major': False, 'units': 4, 'activation': 'tanh', 'recurrent_activation': 'sigmoid', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'recurrent_initializer': {'module': 'keras.initializers', 'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registe

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4)                 256       
                                                                 
 dropout (Dropout)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 261 (1.02 KB)
Trainable params: 261 (1.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# Batch size
batch_size = 32

In [32]:
# Reshape input to be [samples, time steps, features]
X_train  = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [33]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.save("lstm1layer_model.hdf5")

Epoch 1/10
9259/9263 [============================>.] - ETA: 0s - loss: -17.0862 - accuracy: 0.7922

9263/9263 [==============================] - 42s 4ms/step - loss: -17.0938 - accuracy: 0.7922 - val_loss: -32.0410 - val_accuracy: 0.7924
Epoch 2/10
9258/9263 [============================>.] - ETA: 0s - loss: -46.4356 - accuracy: 0.7924

9263/9263 [==============================] - 51s 6ms/step - loss: -46.4421 - accuracy: 0.7924 - val_loss: -60.7937 - val_accuracy: 0.7924
Epoch 3/10
9262/9263 [============================>.] - ETA: 0s - loss: -75.2648 - accuracy: 0.7924

9263/9263 [==============================] - 39s 4ms/step - loss: -75.2670 - accuracy: 0.7924 - val_loss: -89.6334 - val_accuracy: 0.7924
Epoch 4/10
9257/9263 [============================>.] - ETA: 0s - loss: -103.9582 - accuracy: 0.7924

9263/9263 [==============================] - 42s 5ms/step - loss: -103.9666 - accuracy: 0.7924 - val_loss: -118.4192 - val_accuracy: 0.7924
Epoch 5/10
9258/9263 [============================>.] - ETA: 0s - loss: -132.7826 - accuracy: 0.7924

...

9263/9263 [==============================] - 38s 4ms/step - loss: -190.2476 - accuracy: 0.7924 - val_loss: -204.8151 - val_accuracy: 0.7924
Epoch 8/10
9263/9263 [==============================] - ETA: 0s - loss: -219.2233 - accuracy: 0.7924

9263/9263 [==============================] - 37s 4ms/step - loss: -219.2233 - accuracy: 0.7924 - val_loss: -233.5912 - val_accuracy: 0.7924
Epoch 9/10
9256/9263 [============================>.] - ETA: 0s - loss: -247.8340 - accuracy: 0.7923

9263/9263 [==============================] - 37s 4ms/step - loss: -247.8017 - accuracy: 0.7924 - val_loss: -262.3530 - val_accuracy: 0.7924
Epoch 10/10
9259/9263 [============================>.] - ETA: 0s - loss: -276.5818 - accuracy: 0.7924

9263/9263 [==============================] - 37s 4ms/step - loss: -276.5800 - accuracy: 0.7924 - val_loss: -291.1795 - val_accuracy: 0.7924


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict(X_test)
classes_x=np.argmax(y_pred,axis=1)

6176/6176 [==============================] - 17s 3ms/step - loss: -291.1795 - accuracy: 0.7924

Loss: -291.18, Accuracy: 79.24%
6176/6176 [==============================] - 12s 2ms/step
